## 初始化参数
对比特定初始化与随机初始化的区别

In [3]:
import torch
import torch.nn as nn

def initialize_weights(model):
    if isinstance(model, nn.Linear):
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
    elif isinstance(model, nn.Embedding):
        nn.init.normal_(model.weight, mean=0, std=1)
    elif isinstance(model, nn.LayerNorm):
        nn.init.ones_(model.weight)
        nn.init.zeros_(model.bias)

In [4]:
config = GPTConfig()
model = MiniGPT(config).to(torch.bfloat16)
_ = model.apply(initialize_weights)

/data2/anaconda3/envs/python3_10/lib/python3.10/site-packages/torch/nn/modules/module.py:1160: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:305.)
  return t.to(


In [6]:
config = GPTConfig()
dataset_path = "/data2/minigpt/dataset/pretrain/mobvoi_seq_monkey_general_open_corpus.bin"
ds = PretrainBinaryDataset(dataset_path, config.context_length)
sub_ds = Subset(ds, range(1000))
train_loader, eval_loader = create_dataloaders(sub_ds, batch_size=4, train_ratio=0.90)
len(train_loader), len(eval_loader)

total_tokens: 8350205240


(225, 25)

In [ ]:
%%time
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_args = {
    "num_epochs": 1,
    "eval_steps": 500,
}
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
trainer = Trainer(model, optimizer, device, verbose=True)
trainer.set_seed(123)
trainer.set_loader(train_loader, eval_loader)
trainer.train(train_args)

set seed to 123
set train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f7b7449be50>\eval_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f7b785fb550>
start epoch:  1
2024-12-13 12:30:59 cuda:0, train_loss: 7.8323, eval_loss: 7.5761, steps: 500/12250
2024-12-13 12:33:17 cuda:0, train_loss: 7.4436, eval_loss: 7.2764, steps: 1000/12250
2024-12-13 12:35:37 cuda:0, train_loss: 7.1551, eval_loss: 6.9615, steps: 1500/12250
2024-12-13 12:37:56 cuda:0, train_loss: 6.8751, eval_loss: 6.6525, steps: 2000/12250
2024-12-13 12:40:16 cuda:0, train_loss: 6.5821, eval_loss: 6.3639, steps: 2500/12250
2024-12-13 12:42:35 cuda:0, train_loss: 6.3106, eval_loss: 6.0797, steps: 3000/12250
2024-12-13 12:44:54 cuda:0, train_loss: 6.0664, eval_loss: 5.8495, steps: 3500/12250
2024-12-13 12:47:13 cuda:0, train_loss: 5.8419, eval_loss: 5.6428, steps: 4000/12250
2024-12-13 12:49:31 cuda:0, train_loss: 5.6669, eval_loss: 5.4261, steps: 4500/12250
2024-12-13 12:51:50 cuda:0, train_loss: 5.4612, eval_loss: 5.2382, steps: 5000/12250
2024-12-13 12:54:09 cuda:0, train_loss: 5.2547, eval_loss: 5.0406, steps: 5500/12250
2024-12-13 12:56:27 cuda:0, train_loss: 5.0856, eval_loss: 4.8871, steps: 6000/12250
2024-12-13 12:58:45 cuda:0, train_loss: 4.9534, eval_loss: 4.7542, steps: 6500/12250
2024-12-13 13:01:03 cuda:0, train_loss: 4.8042, eval_loss: 4.6657, steps: 7000/12250
2024-12-13 13:03:22 cuda:0, train_loss: 4.7288, eval_loss: 4.5734, steps: 7500/12250
2024-12-13 13:05:44 cuda:0, train_loss: 4.6500, eval_loss: 4.5030, steps: 8000/12250
2024-12-13 13:08:05 cuda:0, train_loss: 4.5745, eval_loss: 4.4449, steps: 8500/12250
2024-12-13 13:10:24 cuda:0, train_loss: 4.5161, eval_loss: 4.4019, steps: 9000/12250
2024-12-13 13:12:42 cuda:0, train_loss: 4.5010, eval_loss: 4.3533, steps: 9500/12250
2024-12-13 13:15:00 cuda:0, train_loss: 4.4435, eval_loss: 4.3193, steps: 10000/12250
2024-12-13 13:17:18 cuda:0, train_loss: 4.4129, eval_loss: 4.2794, steps: 10500/12250
2024-12-13 13:19:36 cuda:0, train_loss: 4.3691, eval_loss: 4.2506, steps: 11000/12250
2024-12-13 13:21:55 cuda:0, train_loss: 4.3471, eval_loss: 4.2261, steps: 11500/12250
2024-12-13 13:24:13 cuda:0, train_loss: 4.3113, eval_loss: 4.1974, steps: 12000/12250
CPU times: user 39min 31s, sys: 17min 5s, total: 56min 37s
Wall time: 56min 35s

很多时候参数初始化的影响可能对于某些任务或结构并不明显，甚至在默认初始化下效果更好，这是业界一个普遍的现象。